In [ ]:
#Colltecing Data From Reddit

In [10]:
# Import the PRAW library to interact with the Reddit API
import praw  

# Import pandas for data manipulation and analysis
import pandas as pd  


In [11]:
# Install the emoji library to handle emoji removal and processing
!pip install emoji


In [12]:
import emoji
print("emoji package installed successfully.")


emoji package installed successfully.


In [13]:
# Setting up the Reddit API
reddit = praw.Reddit(
    client_id='wNgKgy6sNqNZ3Tiq6pviOg',       # My client ID
    client_secret='TCQNcODtqQ3nE2LiJq89cy91MXYpug',  # My client secret
    user_agent='Xinyi Song'                   # My user agent
)


In [14]:
def remove_emoji(text):
    return emoji.replace_emoji(text, replace='')


In [15]:
def scrape_post_and_comments(submission_url):
    # Access the Reddit submission using the provided URL
    submission = reddit.submission(url=submission_url)
    
    # Retrieve the content of the main post
    post_content = remove_emoji(submission.title + "\n\n" + submission.selftext)
    
    # Extract all comments from the post
    comments = []
    submission.comments.replace_more(limit=None)  # Ensure all comments are fully retrieved
    for comment in submission.comments.list():
        comments.append(remove_emoji(comment.body))  # Remove emojis from each comment
    
    return post_content, comments


In [18]:
# Import pandas for data manipulation and saving data into Excel
import pandas as pd  

# Import random to perform random sampling of comments
import random  

In [19]:
# List of Cho Sang-woo Reddit discussion links
sangwoo_links = [
    'https://www.reddit.com/r/squidgame/comments/qanci1/thoughts_on_sangwoo_as_a_character/',
    'https://www.reddit.com/r/squidgame/comments/q3wvu4/sang_woo_is_disgustingly_calculative_why_does/',
    'https://www.reddit.com/r/squidgame/comments/wtpl6f/sang_woo_is_not_evil_at_all_in_the_game/',
    'https://www.reddit.com/r/squidgame/comments/q8t8w6/people_who_are_hating_on_sangwoo_is_forgetting/',
    'https://www.reddit.com/r/squidgame/comments/11husa8/i_liked_sangwoo/',
    'https://www.reddit.com/r/squidgame/comments/qfkzsq/the_strength_of_sangwoos_moral_character/',
    'https://www.reddit.com/r/squidgame/comments/qg8p5m/sangwoo_wasnt_evil_in_the_games_imo/',
    'https://www.reddit.com/r/squidgame/comments/rduska/thoughts_on_cho_sangwoo/',
    'https://www.reddit.com/r/squidgame/comments/q6s8lp/sang_woo/',
    'https://www.reddit.com/r/squidgame/comments/q4ej1e/whether_you_hate_him_or_not_sang_woo_is_a_well/'
]

In [20]:
# Sample 800 comments globally across all posts
def sample_comments_globally(all_contents, max_comments=800):
    all_comments = []

    # Collect all comments across all posts
    for content in all_contents:
        all_comments.extend(content['comments'])

    # Randomly sample 800 comments if more than 800 exist
    if len(all_comments) > max_comments:
        all_comments = random.sample(all_comments, max_comments)

    return all_comments


In [21]:
# Scrape data and save it to an Excel file
def save_sangwoo_corpus(links, filename):
    all_contents = []

    # Iterate through each link to scrape post content and comments
    for link in links:
        post_content, comments = scrape_post_and_comments(link)
        all_contents.append({'post': post_content, 'comments': comments})

    # Sample 800 comments globally
    sampled_comments = sample_comments_globally(all_contents)

    # Flatten the data for easy export
    flattened_data = []

    # Add post content to the flattened data
    for content in all_contents:
        flattened_data.append({'type': 'post', 'text': content['post']})

    # Add sampled comments to the flattened data
    for comment in sampled_comments:
        flattened_data.append({'type': 'comment', 'text': comment})

    # Convert the flattened data to a DataFrame and save to an Excel file
    df = pd.DataFrame(flattened_data)
    df.to_excel(filename, index=False)


In [22]:
# Save Cho Sang-woo's corpus with a maximum of 800 comments
save_sangwoo_corpus(sangwoo_links, 'Cho_Sang-woo_Reddit_Corpus.xlsx')


In [ ]:
#Reddit Finished, Next AO3

In [ ]:
#Collect Metadata

In [24]:
# To make HTTP requests for accessing story pages
import requests  

# To parse the HTML content of the story pages
from bs4 import BeautifulSoup  

# To organize and save the data into a CSV file
import pandas as pd  


In [ ]:
#Get Metadata Process

In [25]:
def get_metadata(story_id):
    # Construct the URL for the specific story using its ID
    url = f"https://archiveofourown.org/works/{story_id}"
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the title
        title = soup.find('h2', class_='title').text.strip() if soup.find('h2', class_='title') else None
        
        # Extract the author
        author = soup.find('a', rel='author').text.strip() if soup.find('a', rel='author') else None
        
        # Extract the number of kudos (likes)
        kudos = int(soup.find('dd', class_='kudos').text.strip().replace(',', '')) if soup.find('dd', class_='kudos') else 0
        
        # Extract the number of hits (views)
        hits = int(soup.find('dd', class_='hits').text.strip().replace(',', '')) if soup.find('dd', class_='hits') else 0
        
        # Extract the number of bookmarks
        bookmarks = int(soup.find('dd', class_='bookmarks').text.strip().replace(',', '')) if soup.find('dd', class_='bookmarks') else 0
        
        # Extract the rating
        rating = soup.find('dd', class_='rating').text.strip() if soup.find('dd', class_='rating') else None
        
        # Extract the warnings
        warnings = soup.find('dd', class_='warning').text.strip() if soup.find('dd', class_='warning') else None
        
        # Extract the category
        category = soup.find('dd', class_='category').text.strip() if soup.find('dd', class_='category') else None
        
        # Extract relationships
        relationships = [tag.text.strip() for tag in soup.find('dd', class_='relationship').find_all('a')] if soup.find('dd', class_='relationship') else []
        
        # Extract characters
        characters = [tag.text.strip() for tag in soup.find('dd', class_='character').find_all('a')] if soup.find('dd', class_='character') else []
        
        # Extract additional tags
        additional_tags = [tag.text.strip() for tag in soup.find('dd', class_='freeform').find_all('a')] if soup.find('dd', class_='freeform') else []
        
        # Return the extracted metadata
        return {
            'story_id': story_id,
            'title': title,
            'author': author,
            'kudos': kudos,
            'hits': hits,
            'bookmarks': bookmarks,
            'rating': rating,
            'warnings': warnings,
            'category': category,
            'relationships': relationships,
            'characters': characters,
            'additional_tags': additional_tags
        }
    else:
        # Return None if the request fails
        return None


In [27]:
#Process Multiple Story IDs

def main_metadata(story_ids, character_name):
    data = []
    # Loop through each story ID to collect its metadata
    for story_id in story_ids:
        story_data = get_metadata(story_id)
        if story_data:
            data.append(story_data)
    
    # Save the collected metadata to a CSV file
    df = pd.DataFrame(data)
    file_name = f'{character_name}_AO3_metadata.csv'
    df.to_csv(file_name, index=False, encoding='utf-8-sig')
    print(f"Metadata saved to {file_name}")


In [28]:
#List of Sang-woo Story IDs
cho_sangwoo_ids = [
    34127974, 34079449, 34448059, 34041061, 34133302, 34203805,
    34387306, 34297090, 34406347, 34139971, 34298500, 34130629,
    34265242, 34444378, 34316797, 34089418, 34339738, 34269919,
    34199386, 34432015, 34426993, 34065412, 34287916, 34410592,
    34379929, 34351207, 34280716, 34080937, 34121575, 34294075
]


In [29]:
# Collect and save metadata for Cho Sang-woo stories
main_metadata(cho_sangwoo_ids, 'Cho_Sang-woo')


Metadata saved to Cho_Sang-woo_AO3_metadata.csv


In [ ]:
#Get full text data

In [30]:
# Get full text from AO3
def get_full_text(url):
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the title
        title = soup.find('h2', class_='title').text.strip() if soup.find('h2', class_='title') else None
        
        # Extract the author
        author = soup.find('a', rel='author').text.strip() if soup.find('a', rel='author') else None
        
        # Extract the full story text
        full_text = soup.find('div', class_='userstuff').get_text(strip=True) if soup.find('div', class_='userstuff') else None
        
        # Return the data as a dictionary
        return {
            'url': url,
            'title': title,
            'author': author,
            'full_text': full_text
        }
    else:
        # Return None if the request fails
        return None


In [31]:
# Scrape full text for a list of story URLs
def scrape_full_text(urls, character_name):
    data = []
    # Loop through each URL to collect full text data
    for url in urls:
        story_data = get_full_text(f'https://archiveofourown.org/works/{url}')
        if story_data:
            data.append(story_data)
    
    # Save the collected full text data to a CSV file
    df = pd.DataFrame(data)
    file_name = f'{character_name}_AO3_full_text.csv'
    df.to_csv(file_name, index=False, encoding='utf-8-sig')
    print(f"Full text saved to {file_name}")


In [32]:
# Cho Sang-woo URLs
cho_sangwoo_urls = [
    34127974, 34079449, 34448059, 34041061, 34133302, 
    34203805, 34387306, 34297090, 34406347, 34139971, 
    34298500, 34130629, 34265242, 34444378, 34316797, 
    34089418, 34339738, 34269919, 34199386, 34432015, 
    34426993, 34065412, 34287916, 34410592, 34379929, 
    34351207, 34280716, 34080937, 34121575, 34294075
]

In [33]:
# Scraping full text for Cho Sang-woo's stories
scrape_full_text(cho_sangwoo_urls, 'Cho_Sang-woo')

Full text saved to Cho_Sang-woo_AO3_full_text.csv
